---

# 4-9. 프로젝트: Vocabulary Size를 변경해서 시도해보기
지금까지는 모델을 변경하고, 모델을 조합해서 성능을 올리는 일에 힘썼습니다. 그런데 어쩌면 성능을 높이는 방법은 단순히 모델을 조정하는 일이 한정되지 않을 수 있습니다. 데이터의 전처리는 모델의 성능에 영향을 직접적으로 줍니다. 특히나 Bag of Words를 기반으로 하는 DTM이나 TF-IDF의 경우, 사용하는 단어의 수를 어떻게 결정하느냐에 따라서 성능에 영향을 줄 수 있겠죠.

중요도가 낮은 단어들까지 포함해서 너무 많은 단어를 사용하는 경우에도 성능이 저하될 수 있고, 반대로 너무 적은 단어들을 사용해도 성능이 저하될 수 있습니다. 그리고 이렇게 변화된 단어의 수는 또 어떤 모델을 사용하느냐에 따라 유리할 수도, 불리할 수도 있습니다.

단어의 수에 따라서 모델의 성능이 어떻게 변하는지 테스트해 봅시다.

```
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)
```

앞서 num_words로 사용할 단어의 수를 조정할 수 있다는 것을 배웠습니다. 빈도수가 많은 순서대로 나열했을 때, num_words의 인자로 준 정숫값만큼의 단어를 사용하고 나머지 단어는 전부 <unk>로 처리하는 원리였었죠.

아래의 두 가지 경우에 대해서 지금까지 사용했던 모델들의 정확도를 직접 확인해 보세요.

## 1. 모든 단어 사용
```
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)
```

## 2. 빈도수 상위 5,000개의 단어만 사용
```
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)
```

## 3. 직접 단어 갯수를 설정해서 사용
위 단계에서 5000으로 제시된 num_words를 다양하게 바꾸어 가며 성능을 확인해보세요. 변화된 단어 수에 따른 모델의 성능을 연구해 보세요. 최소 3가지 경우 이상을 실험해 보기를 권합니다.

사용할 모델

나이브 베이즈 분류기, CNB, 로지스틱 회귀, 서포트 벡터 머신, 결정 트리, 랜덤 포레스트, 그래디언트 부스팅 트리, 보팅

## 4. 딥러닝 모델과 비교해 보기
위 과정을 통해 나온 최적의 모델과 단어 수 조건에서, 본인이 선택한 다른 모델을 적용한 결과와 비교해 봅시다. 감정분석 등에 사용했던 RNN이나 1-D CNN 등의 딥러닝 모델 중 하나를 선택해서 오늘 사용했던 데이터셋을 학습해 보고 나오는 결과를 비교해 봅시다. 단, 공정한 비교를 위해 이때 Word2Vec 등의 pretrained model은 사용하지 않도록 합니다.

---

# 모듈 import

In [6]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## 1. 모든 단어 사용

### -1. train, test 분류

In [7]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [8]:
# 데이터 구성 확인
print('훈련 샘플 수 : {}'.format(len(x_train)))
print('테스트 샘플 수 : {}'.format(len(x_test)))

훈련 샘플 수 : 8982
테스트 샘플 수 : 2246


### -2. 원본 뉴스 데이터로 복원(정수 데이터 -> 텍스트)

In [9]:
# 아래와 같이 뉴스 데이터는 '단어'를 key값으로, 고유한 '정수'를 value로 가지는 딕셔너리를 제공함. 이를 word_index에 저장하겠음

word_index = reuters.get_word_index(path='reuters_word_index.json')

In [10]:
# +3 해주기
index_to_word = {index + 3 : word for word, index in word_index.items()}

In [11]:
# 이 데이터의 0, 1, 2 번은 <pad>, <sos>, <unk>라는 자연어 처리를 위한 특별한 토큰들을 위해 맵핑되어진 번호.  
# 그래서 만들어진 index_to_word에 추가적으로 이 작업을 해주어야 진짜 index_to_word가 완성됨

# index_to_word에 숫자 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣어 줌

for index, token in enumerate(('<pad>', '<sos>', '<unk>')):
    index_to_word[index]=token

In [12]:
# index_to_word로 첫번째 훈련용 뉴스 기사를 원래 텍스트로 복원

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


### -3. 전체 훈련용 뉴스 데이터와 테스트 데이터를 텍스트 데이터로 변환

In [13]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded

In [14]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded

### -4. 벡터화 하기

In [15]:
# DTM은 사이킷런의 CountVectorizer()을 통해서 생성할 수 있음
# DTM을 생성하고, DTM의 크기를 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 26506)


### -5. DTM을 사용해 TF-IDF로 변환

In [16]:
# TF-IDF는 사이킷런의 TfidfTrnsformer()을 통해서 생성 가능  
# TF-IDF는 추가적인 전처리를 하지 않는 이상 DTM과 동일한 크기를 가짐

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 26506)


### -6. 사이킷런에서 제공하는 머신러닝 모델은 공통적으로 fit()함수를 통해 학습 가능

#### -6.1 나이브 베이즈 분류기

In [12]:
# 나이브 베이즈 분류기는 사이킷런의 MultinomialNB()를 통해 사용 가능

# 사이킷런이 제공하는 머신러닝 모델들은 공통적으로 fit()이라는 함수를 제공하고 있는데
# 훈련데이터와 해당 훈련 데이터에 대한 레이블을 인자로 사용하면 모델이 이를 학습함

mod = MultinomialNB()
mod.fit(tfidfv, y_train)

MultinomialNB()

In [13]:
# 테스트 데이터에 정확도를 측정하기 위해서는 훈련 데이터와 동일한 전처리를 거쳐야 함.  
# 다시말해 테스트 데이터도 TF-IDF행렬로 변환해주어야 함. 해당 행렬과 predict() 함수를 통해 에측값을 얻어 정확도를 측정함.

x_test_dtm = dtmvector.transform(x_test) # 테스트 데이터를 dtm으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) # dtm을 tf-idf로 변환

predicted1 = mod.predict(tfidfv_test) # 테스트 데이터에 대한 예측
print('정확도: ', accuracy_score(y_test, predicted1)) # 예측값과 실제값 비교

정확도:  0.5997328584149599


#### -6.2 CNB(Complement Naive Bayes Classifier)
#### 나이브 베이즈 분류기를 보완한 것. 데이터의 불균형을 고려하여 가중치를 부여하는 특징이 있음

In [14]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [15]:
predicted2 = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted2)) #예측값과 실제값 비교

정확도: 0.7649154051647373


#### -6.3 로지스틱 회귀(Logistic Regression)(이름은 회귀지만 분류를 수행)
#### 소프트맥스 함수를 사용한 다중 클래스 분류 알고리즘

In [16]:
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [17]:
predicted3 = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted3)) #예측값과 실제값 비교

정확도: 0.813446126447017


#### -6.4 SVM
#### 서포트 벡터(디시전 바운더리에서 가장 가까운 데이터?)만 사용해서 계산랸이 줄어듦

In [18]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [19]:
predicted4 = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted4)) #예측값과 실제값 비교

정확도: 0.7773820124666073


#### -6.5 결정 트리(decision tree)
#### 스무고개 처럼 예/아니오 질문을 통해 학습, 트리계열의 모델들은  고차원이고 희소한 데이터에 대해서는 성능이 별로 없음

In [20]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [21]:
predicted5 = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted5)) #예측값과 실제값 비교

정확도: 0.6211041852181657


#### -6.6 랜덤 포레스트(Random Forest)
#### 결정 트리는 과적합 될 수 있으나 랜덤포레스트는 결정트리보다 오버피팅 가능성이 적음
#### 여러개의 나무들이 오렌지인지 사과인지 예측하여 각자의 결과를 투표로 모아서 예측

In [22]:
# 랜덤 포레스트 훈련
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [23]:
predicted6 = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted6)) #예측값과 실제값 비교

정확도: 0.6544968833481746


#### -6.7 그래디언트 부스팅 트리
#### 여러개의 결정 트리를 묶어 만드는 앙상블 모델. 랜덤 포레스트와 다르게 이전 트리의 오차를 보완하는 방식으로 순차적으로 트리를 만듦
#### 일반적으로 1-5정도의 깊지 않은 트리만 사용하기 때문에 메모리도 적게 사용하고 예측도 빠름

In [24]:
# 약 12분 정도 소요 됨
grbt = GradientBoostingClassifier(random_state=0, verbose=3) # verbose=3
grbt.fit(tfidfv, y_train)

      Iter       Train Loss   Remaining Time 
         1       12845.1851           13.60m
         2   689466281.7264           13.37m
         3 6884613407215.1387           13.39m
         4 5935818824582923550720.0000           13.16m
         5 9040644020576697454476775298309462042959687372080133879675061827235923374847613060426279764553817024942513812241979907601155443654656.0000           12.88m
         6 9040644020576697454476775298309462042959687372080133879675061827235923374847613060426279764553817024942513812241979907601155443654656.0000           12.56m
         7 9040644020576697454476775298309462042959687372080133879675061827235923374847613060426279764553817024942513812241979907601155443654656.0000           12.42m
         8 9040644020576697454476775298309462042959687372080133879675061827235923374847613060426279764553817024942513812241979907601155443654656.0000           12.29m
         9 9040644020576697454476775298309462042959687372080133879675061827235923374847613060

        51 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            6.27m
        52 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            6.13m
        53 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            5.99m
        54 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            5.87m
        55 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            5.74m
        56 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852

        97 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000           22.95s
        98 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000           15.31s
        99 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            7.65s
       100 71276538223797627407516241100862978529530963044983872457904873785026599868454939389911852427245548475410496325767775571272401790294044479567429632.0000            0.00s


GradientBoostingClassifier(random_state=0, verbose=3)

In [25]:
predicted7 = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted7)) #예측값과 실제값 비교

정확도: 0.7702582368655387


In [26]:
#### -6.8 보팅
#### 하드보팅 : 투표용지를 하나씩 주는 것  
#### 소프트보팅 : 0부터 9중에서 0을 찾는 것이면 0~9 각 레이블마다 예측한 값을 기록하여 그 값을 평균내서 사용하는 것?

In [27]:
#### 로지스틱 회귀, CNB, 그래디언트 부스팅 트리 세가지 사용하여 소프트 보팅으로 성능 비교

In [28]:
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0, verbose=3))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0,
                                                         verbose=3))],
                 n_jobs=-1, voting='soft')

In [29]:
predicted8 = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted8)) #예측값과 실제값 비교

정확도: 0.8187889581478184


In [30]:
print("나이브 베이즈 정확도:", accuracy_score(y_test, predicted1))
print("CNB 정확도:", accuracy_score(y_test, predicted2))
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted3))
print("SVM 정확도:", accuracy_score(y_test, predicted4))
print("결정 트리 정확도:", accuracy_score(y_test, predicted5))
print("랜덤 포레스트 정확도:", accuracy_score(y_test, predicted6))
print("그래디언트 부스팅 트리 정확도:", accuracy_score(y_test, predicted7))
print("보팅 정확도:", accuracy_score(y_test, predicted8))

나이브 베이즈 정확도: 0.5997328584149599
CNB 정확도: 0.7649154051647373
로지스틱 회귀 정확도: 0.813446126447017
SVM 정확도: 0.7773820124666073
결정 트리 정확도: 0.6211041852181657
랜덤 포레스트 정확도: 0.6544968833481746
그래디언트 부스팅 트리 정확도: 0.7702582368655387
보팅 정확도: 0.8187889581478184


## 1. 빈도수 상위 5000 단어 사용

### -1. train, test 분류

In [31]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [32]:
# 데이터 구성 확인
print('훈련 샘플 수 : {}'.format(len(x_train)))
print('테스트 샘플 수 : {}'.format(len(x_test)))

훈련 샘플 수 : 8982
테스트 샘플 수 : 2246


### -2. 원본 뉴스 데이터로 복원(정수 데이터 -> 텍스트)

In [33]:
# 아래와 같이 뉴스 데이터는 '단어'를 key값으로, 고유한 '정수'를 value로 가지는 딕셔너리를 제공함. 이를 word_index에 저장하겠음

word_index = reuters.get_word_index(path='reuters_word_index.json')

In [34]:
# +3 해주기
index_to_word = {index + 3 : word for word, index in word_index.items()}

In [35]:
# 이 데이터의 0, 1, 2 번은 <pad>, <sos>, <unk>라는 자연어 처리를 위한 특별한 토큰들을 위해 맵핑되어진 번호.  
# 그래서 만들어진 index_to_word에 추가적으로 이 작업을 해주어야 진짜 index_to_word가 완성됨

# index_to_word에 숫자 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣어 줌

for index, token in enumerate(('<pad>', '<sos>', '<unk>')):
    index_to_word[index]=token

In [36]:
# index_to_word로 첫번째 훈련용 뉴스 기사를 원래 텍스트로 복원

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


### -3. 전체 훈련용 뉴스 데이터와 테스트 데이터를 텍스트 데이터로 변환

In [37]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded

In [38]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded

### -4. 벡터화 하기

In [39]:
# DTM은 사이킷런의 CountVectorizer()을 통해서 생성할 수 있음
# DTM을 생성하고, DTM의 크기를 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 4867)


### -5. DTM을 사용해 TF-IDF로 변환

In [40]:
# TF-IDF는 사이킷런의 TfidfTrnsformer()을 통해서 생성 가능  
# TF-IDF는 추가적인 전처리를 하지 않는 이상 DTM과 동일한 크기를 가짐

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 4867)


### -6. 사이킷런에서 제공하는 머신러닝 모델은 공통적으로 fit()함수를 통해 학습 가능

#### -6.1 나이브 베이즈 분류기

In [41]:
# 나이브 베이즈 분류기는 사이킷런의 MultinomialNB()를 통해 사용 가능

# 사이킷런이 제공하는 머신러닝 모델들은 공통적으로 fit()이라는 함수를 제공하고 있는데
# 훈련데이터와 해당 훈련 데이터에 대한 레이블을 인자로 사용하면 모델이 이를 학습함

mod = MultinomialNB()
mod.fit(tfidfv, y_train)

MultinomialNB()

In [42]:
# 테스트 데이터에 정확도를 측정하기 위해서는 훈련 데이터와 동일한 전처리를 거쳐야 함.  
# 다시말해 테스트 데이터도 TF-IDF행렬로 변환해주어야 함. 해당 행렬과 predict() 함수를 통해 에측값을 얻어 정확도를 측정함.

x_test_dtm = dtmvector.transform(x_test) # 테스트 데이터를 dtm으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) # dtm을 tf-idf로 변환

predicted1 = mod.predict(tfidfv_test) # 테스트 데이터에 대한 예측
print('정확도: ', accuracy_score(y_test, predicted1)) # 예측값과 실제값 비교

정확도:  0.6731967943009796


#### -6.2 CNB(Complement Naive Bayes Classifier)
#### 나이브 베이즈 분류기를 보완한 것. 데이터의 불균형을 고려하여 가중치를 부여하는 특징이 있음

In [43]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [44]:
predicted2 = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted2)) #예측값과 실제값 비교

정확도: 0.7707034728406055


#### -6.3 로지스틱 회귀(Logistic Regression)(이름은 회귀지만 분류를 수행)
#### 소프트맥스 함수를 사용한 다중 클래스 분류 알고리즘

In [45]:
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [46]:
predicted3 = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted3)) #예측값과 실제값 비교

정확도: 0.8058771148708815


#### -6.4 SVM
#### 서포트 벡터(디시전 바운더리에서 가장 가까운 데이터?)만 사용해서 계산랸이 줄어듦

In [47]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [48]:
predicted4 = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted4)) #예측값과 실제값 비교

정확도: 0.7666963490650045


#### -6.5 결정 트리(decision tree)
#### 스무고개 처럼 예/아니오 질문을 통해 학습, 트리계열의 모델들은  고차원이고 희소한 데이터에 대해서는 성능이 별로 없음

In [49]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [50]:
predicted5 = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted5)) #예측값과 실제값 비교

정확도: 0.6179875333926982


#### -6.6 랜덤 포레스트(Random Forest)
#### 결정 트리는 과적합 될 수 있으나 랜덤포레스트는 결정트리보다 오버피팅 가능성이 적음
#### 여러개의 나무들이 오렌지인지 사과인지 예측하여 각자의 결과를 투표로 모아서 예측

In [51]:
# 랜덤 포레스트 훈련
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [52]:
predicted6 = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted6)) #예측값과 실제값 비교

정확도: 0.701246660730187


#### -6.7 그래디언트 부스팅 트리
#### 여러개의 결정 트리를 묶어 만드는 앙상블 모델. 랜덤 포레스트와 다르게 이전 트리의 오차를 보완하는 방식으로 순차적으로 트리를 만듦
#### 일반적으로 1-5정도의 깊지 않은 트리만 사용하기 때문에 메모리도 적게 사용하고 예측도 빠름

In [53]:
# 약 12분 정도 소요 됨
grbt = GradientBoostingClassifier(random_state=0, verbose=3) # verbose=3
grbt.fit(tfidfv, y_train)

      Iter       Train Loss   Remaining Time 
         1       13200.9090           11.46m
         2 19141531432.3860           11.41m
         3 1018136056482137345428864037762575643679300980036667356656533462569269067776.0000           11.33m
         4 5793438208146438552371750463374243580454084126740516228669142075250525372030044080110043136.0000           11.22m
         5 3979630484381169793867261128601856110593550227404752540847489843154493525639763256294740889013872088305157670725524817116093080321688237768704.0000           10.85m
         6 3979630484381169793867261128601856110593550227404752540847489843154493525639763256294740889013872088305157670725524817116093080321688237768704.0000           10.54m
         7 3979630484381169793867261128601856110593550227404752540847489843154493525639763256294740889013872088305157670725524817116093080321688237768704.0000           10.29m
         8 397963048438116979386726112860185611059355022740475254084748984315449352563976325629474088

        50 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            5.68m
        51 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            5.56m
        52 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            5.43m
        53 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            5.31m
        54 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            5.20m
        55 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122

        97 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000           20.41s
        98 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000           13.61s
        99 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            6.80s
       100 3979630484381173178473821334674682744399927355192476462279466620274336263041595061252392018927962713273915122067783236783098648441282755559424.0000            0.00s


GradientBoostingClassifier(random_state=0, verbose=3)

In [54]:
predicted7 = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted7)) #예측값과 실제값 비교

정확도: 0.767586821015138


#### -6.8 보팅
#### 하드보팅 : 투표용지를 하나씩 주는 것  
#### 소프트보팅 : 0부터 9중에서 0을 찾는 것이면 0~9 각 레이블마다 예측한 값을 기록하여 그 값을 평균내서 사용하는 것?

#### 로지스틱 회귀, CNB, 그래디언트 부스팅 트리 세가지 사용하여 소프트 보팅으로 성능 비교

In [57]:
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0, verbose=3))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0,
                                                         verbose=3))],
                 n_jobs=-1, voting='soft')

In [58]:
predicted8 = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted8)) #예측값과 실제값 비교

정확도: 0.8161175422974176


In [59]:
print("나이브 베이즈 정확도:", accuracy_score(y_test, predicted1))
print("CNB 정확도:", accuracy_score(y_test, predicted2))
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted3))
print("SVM 정확도:", accuracy_score(y_test, predicted4))
print("결정 트리 정확도:", accuracy_score(y_test, predicted5))
print("랜덤 포레스트 정확도:", accuracy_score(y_test, predicted6))
print("그래디언트 부스팅 트리 정확도:", accuracy_score(y_test, predicted7))
print("보팅 정확도:", accuracy_score(y_test, predicted8))

나이브 베이즈 정확도: 0.6731967943009796
CNB 정확도: 0.7707034728406055
로지스틱 회귀 정확도: 0.8058771148708815
SVM 정확도: 0.7666963490650045
결정 트리 정확도: 0.6179875333926982
랜덤 포레스트 정확도: 0.701246660730187
그래디언트 부스팅 트리 정확도: 0.767586821015138
보팅 정확도: 0.8161175422974176


## 3. 직접 단어 갯수 설정해서 사용(15000)

### -1. train, test 분류

In [60]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=15000, test_split=0.2)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [61]:
# 데이터 구성 확인
print('훈련 샘플 수 : {}'.format(len(x_train)))
print('테스트 샘플 수 : {}'.format(len(x_test)))

훈련 샘플 수 : 8982
테스트 샘플 수 : 2246


### -2. 원본 뉴스 데이터로 복원(정수 데이터 -> 텍스트)

In [62]:
# 아래와 같이 뉴스 데이터는 '단어'를 key값으로, 고유한 '정수'를 value로 가지는 딕셔너리를 제공함. 이를 word_index에 저장하겠음

word_index = reuters.get_word_index(path='reuters_word_index.json')

In [63]:
# +3 해주기
index_to_word = {index + 3 : word for word, index in word_index.items()}

In [64]:
# 이 데이터의 0, 1, 2 번은 <pad>, <sos>, <unk>라는 자연어 처리를 위한 특별한 토큰들을 위해 맵핑되어진 번호.  
# 그래서 만들어진 index_to_word에 추가적으로 이 작업을 해주어야 진짜 index_to_word가 완성됨

# index_to_word에 숫자 0은 <pad>, 1은 <sos>, 2는 <unk>를 넣어 줌

for index, token in enumerate(('<pad>', '<sos>', '<unk>')):
    index_to_word[index]=token

In [65]:
# index_to_word로 첫번째 훈련용 뉴스 기사를 원래 텍스트로 복원

print(' '.join([index_to_word[index] for index in x_train[0]]))

<sos> <unk> <unk> said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


### -3. 전체 훈련용 뉴스 데이터와 테스트 데이터를 텍스트 데이터로 변환

In [66]:
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded

In [67]:
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded

### -4. 벡터화 하기

In [68]:
# DTM은 사이킷런의 CountVectorizer()을 통해서 생성할 수 있음
# DTM을 생성하고, DTM의 크기를 확인

dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 14227)


### -5. DTM을 사용해 TF-IDF로 변환

In [69]:
# TF-IDF는 사이킷런의 TfidfTrnsformer()을 통해서 생성 가능  
# TF-IDF는 추가적인 전처리를 하지 않는 이상 DTM과 동일한 크기를 가짐

tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 14227)


### -6. 사이킷런에서 제공하는 머신러닝 모델은 공통적으로 fit()함수를 통해 학습 가능

#### -6.1 나이브 베이즈 분류기

In [70]:
# 나이브 베이즈 분류기는 사이킷런의 MultinomialNB()를 통해 사용 가능

# 사이킷런이 제공하는 머신러닝 모델들은 공통적으로 fit()이라는 함수를 제공하고 있는데
# 훈련데이터와 해당 훈련 데이터에 대한 레이블을 인자로 사용하면 모델이 이를 학습함

mod = MultinomialNB()
mod.fit(tfidfv, y_train)

MultinomialNB()

In [71]:
# 테스트 데이터에 정확도를 측정하기 위해서는 훈련 데이터와 동일한 전처리를 거쳐야 함.  
# 다시말해 테스트 데이터도 TF-IDF행렬로 변환해주어야 함. 해당 행렬과 predict() 함수를 통해 에측값을 얻어 정확도를 측정함.

x_test_dtm = dtmvector.transform(x_test) # 테스트 데이터를 dtm으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) # dtm을 tf-idf로 변환

predicted1 = mod.predict(tfidfv_test) # 테스트 데이터에 대한 예측
print('정확도: ', accuracy_score(y_test, predicted1)) # 예측값과 실제값 비교

정확도:  0.6331255565449688


#### -6.2 CNB(Complement Naive Bayes Classifier)
#### 나이브 베이즈 분류기를 보완한 것. 데이터의 불균형을 고려하여 가중치를 부여하는 특징이 있음

In [72]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [73]:
predicted2 = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted2)) #예측값과 실제값 비교

정확도: 0.7720391807658059


#### -6.3 로지스틱 회귀(Logistic Regression)(이름은 회귀지만 분류를 수행)
#### 소프트맥스 함수를 사용한 다중 클래스 분류 알고리즘

In [74]:
lr = LogisticRegression(C=10000, penalty='l2')
lr.fit(tfidfv, y_train)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10000)

In [75]:
predicted3 = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted3)) #예측값과 실제값 비교

정확도: 0.8125556544968834


#### -6.4 SVM
#### 서포트 벡터(디시전 바운더리에서 가장 가까운 데이터?)만 사용해서 계산랸이 줄어듦

In [76]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=500, dual=False)
lsvc.fit(tfidfv, y_train)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1000, dual=False, max_iter=500, penalty='l1')

In [77]:
predicted4 = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted4)) #예측값과 실제값 비교

정확도: 0.769813000890472


#### -6.5 결정 트리(decision tree)
#### 스무고개 처럼 예/아니오 질문을 통해 학습, 트리계열의 모델들은  고차원이고 희소한 데이터에 대해서는 성능이 별로 없음

In [78]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [79]:
predicted5 = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted5)) #예측값과 실제값 비교

정확도: 0.6193232413178985


#### -6.6 랜덤 포레스트(Random Forest)
#### 결정 트리는 과적합 될 수 있으나 랜덤포레스트는 결정트리보다 오버피팅 가능성이 적음
#### 여러개의 나무들이 오렌지인지 사과인지 예측하여 각자의 결과를 투표로 모아서 예측

In [80]:
# 랜덤 포레스트 훈련
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [81]:
predicted6 = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted6)) #예측값과 실제값 비교

정확도: 0.6714158504007124


#### -6.7 그래디언트 부스팅 트리
#### 여러개의 결정 트리를 묶어 만드는 앙상블 모델. 랜덤 포레스트와 다르게 이전 트리의 오차를 보완하는 방식으로 순차적으로 트리를 만듦
#### 일반적으로 1-5정도의 깊지 않은 트리만 사용하기 때문에 메모리도 적게 사용하고 예측도 빠름

In [82]:
# 약 12분 정도 소요 됨
grbt = GradientBoostingClassifier(random_state=0, verbose=3) # verbose=3
grbt.fit(tfidfv, y_train)

      Iter       Train Loss   Remaining Time 
         1       12867.7588           12.36m
         2   730517195.5810           11.71m
         3 52426240961.5098           11.26m
         4 572955846448707528749563296618507038765457410545942528.0000           11.00m
         5 1761988519418287540202658257396981492463494756859736592051509137556573860600479744.0000           10.80m
         6 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000           10.63m
         7 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000           10.47m
         8 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000           10.33m
         9 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000 

        56 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000            5.40m
        57 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000            5.28m
        58 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000            5.16m
        59 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000            5.04m
        60 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000            4.92m
        61 1759787442570566016765056566811045247797243136933711134816622164432437917777407526013667765525766091985683072831844161421312.0000            4.79m
        62 17597874425705660167650565668110452477972

GradientBoostingClassifier(random_state=0, verbose=3)

In [83]:
predicted7 = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted7)) #예측값과 실제값 비교

정확도: 0.7707034728406055


#### -6.8 보팅
#### 하드보팅 : 투표용지를 하나씩 주는 것  
#### 소프트보팅 : 0부터 9중에서 0을 찾는 것이면 0~9 각 레이블마다 예측한 값을 기록하여 그 값을 평균내서 사용하는 것?

#### 로지스틱 회귀, CNB, 그래디언트 부스팅 트리 세가지 사용하여 소프트 보팅으로 성능 비교

In [86]:
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0, verbose=3))
], voting='soft', n_jobs=-1)
voting_classifier.fit(tfidfv, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=10000)),
                             ('cb', ComplementNB()),
                             ('grbt',
                              GradientBoostingClassifier(random_state=0,
                                                         verbose=3))],
                 n_jobs=-1, voting='soft')

In [87]:
predicted8 = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted8)) #예측값과 실제값 비교

정확도: 0.8165627782724845


In [88]:
print("나이브 베이즈 정확도:", accuracy_score(y_test, predicted1))
print("CNB 정확도:", accuracy_score(y_test, predicted2))
print("로지스틱 회귀 정확도:", accuracy_score(y_test, predicted3))
print("SVM 정확도:", accuracy_score(y_test, predicted4))
print("결정 트리 정확도:", accuracy_score(y_test, predicted5))
print("랜덤 포레스트 정확도:", accuracy_score(y_test, predicted6))
print("그래디언트 부스팅 트리 정확도:", accuracy_score(y_test, predicted7))
print("보팅 정확도:", accuracy_score(y_test, predicted8))

나이브 베이즈 정확도: 0.6331255565449688
CNB 정확도: 0.7720391807658059
로지스틱 회귀 정확도: 0.8125556544968834
SVM 정확도: 0.769813000890472
결정 트리 정확도: 0.6193232413178985
랜덤 포레스트 정확도: 0.6714158504007124
그래디언트 부스팅 트리 정확도: 0.7707034728406055
보팅 정확도: 0.8165627782724845


## 4. 딥러닝 모델과 비교해 보기
위 과정을 통해 나온 최적의 모델과 단어 수 조건에서, 본인이 선택한 다른 모델을 적용한 결과와 비교해 봅시다. 감정분석 등에 사용했던 RNN이나 1-D CNN 등의 딥러닝 모델 중 하나를 선택해서 오늘 사용했던 데이터셋을 학습해 보고 나오는 결과를 비교해 봅시다. 단, 공정한 비교를 위해 이때 Word2Vec 등의 pretrained model은 사용하지 않도록 합니다.

#### 내가 뭘 적용해서 해보기에는 나는 아직도 이걸 어떻게 하는지 모르겠다. 노드를 보고 몇 개 수정하거나 복사하여 붙여넣는 것은 할 수 있지만 이렇게 개인이 새롭게 해봐야 하는 내용이 나오면 어떻게 하는지 이해도 잘 안되고 잘 모르겠다.

# 총평

## 루브릭 평가내용
1. 분류 모델의 accuracy가 기준 이상 높게 나왔는가?  
3가지 단어 개수에 대해 8가지 머신러닝 기법을 적용하여 그중 최적의 솔루션을 도출하였다.
- 노드에서 제공해준 것에 대해선 다 실행해봤고 보팅이 제일 좋은 결과를 얻었다.


2. 분류 모델의 F1 score가 기준 이상 높게 나왔는가?  
Vocabulary size에 따른 각 머신러닝 모델의 성능변화 추이를 살피고, 해당 머신러닝 알고리즘의 특성에 근거해 원인을 분석하였다.
- 결과가 좋은 것부터 나쁜 것까지 확인할 수 있었지만 근거해 원인을 분석하기엔 아직 잘 모르겠다.


3. 생성모델의 metric(BLEU 등) 기준 이상 높은 성능이 확인되었는가?  
동일한 데이터셋과 전처리 조건으로 딥러닝 모델의 성능과 비교하여 결과에 따른 원인을 분석하였다.
- 어떤 이상이 높은 성능인지 모르겠지만 80%이상이 나왔으니? 만족할만한 것 같다. 원인을 분석하기엔 실력이 부족하다.